In [1]:
#build_medicalgraph
import os
import json
from py2neo import Graph,Node

# 1录入

In [2]:
def get_quantities(i,j,sen:list):
    """
    item[i,j]是药品
    """
    for item in sen:
        if item[2] == "quantities":
            if item[4] == i and item[5] == j:
                if item[3] == "QUA":
                    return item[0]
                elif item[6] == "QUA":
                    return item[1]
            elif item[7] == i and item[8] == j:
                if item[3] == "QUA":
                    return item[0]
                elif item[6] == "QUA":
                    return item[1]
    return "N/A"

In [3]:
def get_idx(i,j,sen:list):
    """
    item[i,j]是USE
    """
    # 构造sen中use的set
    set_sen = set()
    for item in sen:
        if item[3] == "USE":
            set_sen.add((item[4],item[5]))
        if item[6] == "USE":
            set_sen.add((item[7],item[8]))
    
    set_sen = list(set_sen)
    set_sen = sorted(set_sen,key=lambda t:int(t[0]))  
    
    #print(set_sen)
    for k in range(len(set_sen)):
        if set_sen[k][0] == i and set_sen[k][1] == j:
            return i
    print("error!!")
    return "error"
    

In [4]:
data_rel = []
sen = []
for line in  open("data_res/triple.txt",encoding= "utf-8"):
    line = line.split()
    if len(line) >0:
        sen.append(line)
    elif len(line) == 0:
        data_rel.append(sen)
        sen = []

In [5]:
class MedicalGraph:
    def __init__(self):
        #cur_dir = '/'.join(os.path.abspath(__file__).split('/')[:-1])
        #self.data_path = os.path.join(dir_path + 'KG_built/')
        self.g = Graph(
            host="localhost",  # neo4j 搭载服务器的ip地址，ifconfig可获取到
            http_port=7474,  # neo4j 服务器监听的端口号
            user="neo4j",  # 数据库user name，如果没有更改过，应该是neo4j
            password="password")
        
    '''读取文件'''
    def read_nodes(self):        
        # 共4类节点
        names = [] # 方名
        drugs = [] # 草药
        uses = [] # 用法
        symptoms = [] # 症状
        
        # 构建节点实体关系
        rels_name_dru = [] #　方名、草药之间的关系（关系的属性是药量）
        rels_name_anotherName = [] # 方名、又方之间的关系
        rels_name_use = [] # 方名、用法之间的关系
        rels_name_sym = [] # 方名、症状之间的关系
        rels_use_sym = [] # 用法、症状之间的关系
        
        for line in open('data_res/DRU_CORPUS.txt',encoding="utf-8"):
            line = line.strip()
            drugs.append(line)
            
        for line in open('data_res/NAM_CORPUS.txt',encoding="utf-8"):
            line = line.strip()
            names.append(line)
        
        for line in open('data_res/SYM_CORPUS.txt',encoding="utf-8"):
            line = line.strip()
            symptoms.append(line)
            
        for line in open('data_res/USE_CORPUS.txt',encoding="utf-8"):
            line = line.strip()
            uses.append(line)
        
        data_rel = []
        sen = []
        for line in  open("data_res/triple.txt",encoding= "utf-8"):
            line = line.split()
            if len(line) >0:
                sen.append(line)
            elif len(line) == 0:
                data_rel.append(sen)
                sen = []
        # 构建节点实体关系
        rels_name_dru = [] #　方名、草药之间的关系（关系的属性是药量）
        rels_name_anotherName = [] # 方名、又方之间的关系
        rels_name_use = [] # 方名、用法之间的关系
        rels_name_sym = [] # 方名、症状之间的关系
        rels_use_sym = [] # 用法、症状之间的关系


        # 文件中存有：【NAM USE uses】【NAM SYM cure】【USE SYM match】【NAM DRU contain】【DRU QUAquantities】
        for sen in data_rel:
            for item in sen:
                #  【NAM DRU contain】【quantities】
                if item[2] == "contain":
                    if item[3] == "NAM" and item[6] == "DRU":
                        q = get_quantities(item[7],item[8],sen)###
                        rels_name_dru.append([item[0],item[1],q])###
                    else:
                        print("error:",item)
                #  【NAM USE uses】
                elif item[2] == "uses":
                    if item[3] == "NAM" and item[6] == "USE":
                        idx = get_idx(item[7],item[8],sen)###
                        rels_name_use.append([item[0],item[1],idx])###
                    else:
                        print("error:",item)
                #  【NAM SYM cure】
                elif item[2] == "cure":
                    if item[3] == "NAM" and item[6] == "SYM":
                        rels_name_sym.append([item[0],item[1]])
                    else:
                        print("error:",item)
                # 【USE SYM match】
                elif item[2] == "match":
                    if item[3] == "USE" and item[6] == "SYM":
                        rels_use_sym.append([item[0],item[1]])
                    else: 
                        print("error:",item)
                # 【NAM ANORTHERNAME unknown】
                elif item[3] == "NAM" and item[6] == "NAM":
                    if item[0][:2] == "又方" or item[1][:2] == "又方":
                        rels_name_anotherName.append([item[0],item[1]])
                               
        return set(names), set(drugs), set(uses), set(symptoms), rels_name_dru, rels_name_anotherName, rels_name_use, rels_name_sym, rels_use_sym
        
    
    '''建立节点'''
    def create_node(self, label, nodes):
        count = 0
        for node_name in nodes:
            node = Node(label, name=node_name)
            self.g.create(node)
            count += 1
            #print(count, len(nodes))
        return

    '''创建知识图谱实体节点类型schema'''
    def create_graphnodes(self):
        Names, Drugs, Uses, Symptoms, rels_name_dru, rels_name_anotherName, rels_name_use, rels_name_sym, rels_use_sym  = self.read_nodes()
        self.create_node('Name', Names)
        self.create_node('Drug', Drugs)
        self.create_node('Use', Uses)
        self.create_node('Symptom', Symptoms)
        return


    '''创建实体关系边'''
    def create_graphrels(self):
        Names, Drugs, Uses, Symptoms, rels_name_dru, rels_name_anotherName, rels_name_use, rels_name_sym, rels_use_sym  = self.read_nodes()

        self.create_relationship_withquantities('Name', 'Drug', rels_name_dru, '包含')
        self.create_relationship_with_id('Name', 'Use', rels_name_use , '使用方法')
        self.create_relationship('Name', 'Name', rels_name_anotherName, '又方')
        self.create_relationship('Name', 'Symptom', rels_name_sym,'治疗')
        self.create_relationship('Use', 'Symptom', rels_use_sym,'对应症状')
        """
        shiti = ["Name","Drug","Use","Symptom"]
        cypher1 = ["match(a)-[r]-(b:"+ i+ ") where b.name='' delete r" for i in shiti]
        cypher2 = ["match (n:"+ i +") where n.name='' delete n" for i in shiti]
        for i in cypher1:
            self.g.run(i)

        for i in cypher2:
            self.g.run(i)
        """
    '''创建实体关联边'''
    def create_relationship(self, start_node, end_node, edges,rel_type):
        count = 0
        # 去重处理
        set_edges = []
        for edge in edges:
            set_edges.append('###'.join(edge))
        all = len(set(set_edges))
        for edge in set(set_edges):
            edge = edge.split('###')
            p = edge[0]
            q = edge[1]
            
            query = "match(p:%s),(q:%s) where p.name='%s'and q.name='%s' create (p)-[rel:%s]->(q)" % (
                start_node, end_node, p, q, rel_type)
            try:
                self.g.run(query)
                count += 1
                #print(rel_type, count, all)
            except Exception as e:
                print(e)
        return

    '''创建实体关联边'''
    def create_relationship_withquantities(self, start_node, end_node, edges, rel_type):
        """
        edges = [a,b,c]
        a b 是实体
        c是关系的名字
        """
        count = 0
        # 去重处理
        set_edges = []
        for edge in edges:
            set_edges.append('###'.join(edge))# entity1###entity2
        all = len(set(set_edges))
        
        for edge in set(set_edges):
            edge = edge.split('###')
            p = edge[0]
            q = edge[1]
            quantiti = edge[2]
            query = "match(p:%s),(q:%s) where p.name='%s'and q.name='%s' create (p)-[rel:%s{分量:'%s'}]->(q)" % (
                start_node, end_node, p, q, rel_type, quantiti)
            try:
                self.g.run(query)
                count += 1
                #print(rel_type, count, all)
            except Exception as e:
                print(e)
        return

    '''创建实体关联边'''
    def create_relationship_with_id(self, start_node, end_node, edges, rel_type):
        """
        edges = [a,b,c]
        a b 是实体
        c是关系的顺序编号
        """
        count = 0
        # 去重处理
        set_edges = []
        for edge in edges:
            set_edges.append('###'.join(edge))# entity1###entity2
        all = len(set(set_edges))
        
        for edge in set(set_edges):
            edge = edge.split('###')
            p = edge[0]
            q = edge[1]
            idx = edge[2]
            query = "match(p:%s),(q:%s) where p.name='%s'and q.name='%s' create (p)-[rel:%s{起始索引:'%s',最大索引:149}]->(q)" % (
                start_node, end_node, p, q, rel_type, idx)
            try:
                self.g.run(query)
                count += 1
                #print(rel_type, count, all)
            except Exception as e:
                print(e)
        return
    
    def run(self,str_run):
        self.g.run(str_run)
        return

In [6]:
if __name__ == '__main__':
    handler = MedicalGraph()
    handler.create_graphnodes()
    handler.create_graphrels()